In [ ]:
LSTM without one hot

In [2]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import numpy as np
from util.NNmodel.LSTM import TaxiLSTM
import util.NNmodel.ConvLSTM
from util.preprocess import TaxiDataset

#training config
epochs = 30
batch_size = 72
learning_rate=0.001

# Device config
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#train the model
def train_model(model, dataloader):
    model.train()
    all_loss = []
    #for _ in tqdm(range(total_step), ncols=80):
    for i, (features, labels, locations, times) in enumerate(train_dataloader):
        features = features.to(torch.float32).to(device)
        labels = labels.to(torch.float32).to(device)
        locations = locations.to(torch.float32).to(device)
        times = times.to(torch.float32).to(device)

        # Forward pass
        outputs = model(features, locations, times)
        loss = torch.sqrt(criterion(outputs, labels))
        all_loss.append(loss.item())

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    return np.mean(all_loss)

# Test the model
def test_model(model, dataloader):
    model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
    all_loss = []
    with torch.no_grad():
        for features, labels, locations, times in dataloader:
            features = features.to(torch.float32).to(device)
            labels = labels.to(torch.float32).to(device)
            locations = locations.to(torch.float32).to(device)
            times = times.to(torch.float32).to(device)

            outputs = model(features, locations, times)
            loss = torch.sqrt(criterion(outputs, labels)).item()
            all_loss.append(loss)
        return np.mean(all_loss)

#read data
train_data = np.load('./train.npz')
train_features = train_data['x']#features
train_labels = train_data['y']#labels
train_locations = train_data['locations']#locations
train_times = train_data['times']#times
val_data = np.load('./val.npz')
val_features = val_data['x']#features
val_labels = val_data['y']#labels
val_locations = val_data['locations']#locations
val_times = val_data['times']#times

# dataset
train_dataset = TaxiDataset(train_features, train_labels, train_locations, train_times)
val_dataset = TaxiDataset(val_features, val_labels, val_locations, val_times)
# dataloader
train_dataloader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=True)

# initialize model
model = TaxiLSTM().to(device)

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
for epoch in range(epochs):
    train_loss = train_model(model, train_dataloader)
    val_loss = test_model(model, val_dataloader)
    print(f'Epoch [{epoch+1}/{epochs}]: The traing set Root Main Square Error loss: {np.mean(train_loss)}. The RMSE of validation set is {val_loss}')
#with open('./time.txt',"a") as f:
#    for n in times:
#        f.write(str(n))
#    f.close()

Epoch [1/80]: The traing set Root Main Square Error loss: 7.520054617643356. The RMSE of validation set is 5.891659136772156
Epoch [2/80]: The traing set Root Main Square Error loss: 5.312083241462708. The RMSE of validation set is 5.649008751869202
Epoch [3/80]: The traing set Root Main Square Error loss: 4.878128673076629. The RMSE of validation set is 5.256977566719055
Epoch [4/80]: The traing set Root Main Square Error loss: 4.697097569942474. The RMSE of validation set is 5.038237740516663
Epoch [5/80]: The traing set Root Main Square Error loss: 4.5781216039657595. The RMSE of validation set is 5.087163338661194
Epoch [6/80]: The traing set Root Main Square Error loss: 4.477439803123474. The RMSE of validation set is 5.513822565078735
Epoch [7/80]: The traing set Root Main Square Error loss: 4.418442833662033. The RMSE of validation set is 5.990305102348327
Epoch [8/80]: The traing set Root Main Square Error loss: 4.388113314390183. The RMSE of validation set is 5.001856076240539

Epoch [66/80]: The traing set Root Main Square Error loss: 3.6852544012069703. The RMSE of validation set is 4.486823302268982
Epoch [67/80]: The traing set Root Main Square Error loss: 3.6795943977832795. The RMSE of validation set is 4.438393665313721
Epoch [68/80]: The traing set Root Main Square Error loss: 3.666610196709633. The RMSE of validation set is 4.475916582107544
Epoch [69/80]: The traing set Root Main Square Error loss: 3.641891600370407. The RMSE of validation set is 4.486837797164917
Epoch [70/80]: The traing set Root Main Square Error loss: 3.640384788274765. The RMSE of validation set is 4.57298078250885
Epoch [71/80]: The traing set Root Main Square Error loss: 3.6443355395793913. The RMSE of validation set is 4.475708111763001
Epoch [72/80]: The traing set Root Main Square Error loss: 3.619988327026367. The RMSE of validation set is 4.639895482063293
Epoch [73/80]: The traing set Root Main Square Error loss: 3.6472877115011215. The RMSE of validation set is 4.43644

In [ ]:
CNN

In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import numpy as np
from util.NNmodel.LSTM import TaxiLSTM
from util.NNmodel.CNN import TaxiCNN
import util.NNmodel.ConvLSTM 
from util.preprocess import TaxiDataset

#training config
epochs = 30
batch_size = 72
learning_rate=0.001

# Device config
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#train the model
def train_model(model, dataloader):
    model.train()
    all_loss = []
    #for _ in tqdm(range(total_step), ncols=80):
    for i, (features, labels, locations, times) in enumerate(train_dataloader):
        features = features.to(torch.float32).to(device)
        labels = labels.to(torch.float32).to(device)
        locations = locations.to(torch.float32).to(device)
        times = times.to(torch.float32).to(device)

        # Forward pass
        outputs = model(features, locations, times)
        loss = torch.sqrt(criterion(outputs, labels))
        all_loss.append(loss.item())

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    return np.mean(all_loss)

# Test the model
def test_model(model, dataloader):
    model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
    all_loss = []
    with torch.no_grad():
        for features, labels, locations, times in dataloader:
            features = features.to(torch.float32).to(device)
            labels = labels.to(torch.float32).to(device)
            locations = locations.to(torch.float32).to(device)
            times = times.to(torch.float32).to(device)

            outputs = model(features, locations, times)
            loss = torch.sqrt(criterion(outputs, labels)).item()
            all_loss.append(loss)
        return np.mean(all_loss)

#read data
train_data = np.load('./train.npz')
train_features = train_data['x']#features
train_labels = train_data['y']#labels
train_locations = train_data['locations']#locations
train_times = train_data['times']#times
val_data = np.load('./val.npz')
val_features = val_data['x']#features
val_labels = val_data['y']#labels
val_locations = val_data['locations']#locations
val_times = val_data['times']#times

# dataset
train_dataset = TaxiDataset(train_features, train_labels, train_locations, train_times)
val_dataset = TaxiDataset(val_features, val_labels, val_locations, val_times)
# dataloader
train_dataloader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=True)

# initialize model
model = TaxiCNN().to(device)

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
for epoch in range(epochs):
    train_loss = train_model(model, train_dataloader)
    val_loss = test_model(model, val_dataloader)
    print(f'Epoch [{epoch+1}/{epochs}]: The traing set Root Main Square Error loss: {np.mean(train_loss)}. The RMSE of validation set is {val_loss}')
#with open('./time.txt',"a") as f:
#    for n in times:
#        f.write(str(n))
#    f.close()

Epoch [1/30]: The traing set Root Main Square Error loss: 6.216972737073898. The RMSE of validation set is 5.2456781253814695
Epoch [2/30]: The traing set Root Main Square Error loss: 4.907362941741943. The RMSE of validation set is 4.9295841436386105
Epoch [3/30]: The traing set Root Main Square Error loss: 4.645951392173767. The RMSE of validation set is 4.835874801635742
Epoch [4/30]: The traing set Root Main Square Error loss: 4.496503124713898. The RMSE of validation set is 4.870440487861633
Epoch [5/30]: The traing set Root Main Square Error loss: 4.386406612157821. The RMSE of validation set is 4.634033605575562
Epoch [6/30]: The traing set Root Main Square Error loss: 4.3145000245571135. The RMSE of validation set is 4.621151481628418
Epoch [7/30]: The traing set Root Main Square Error loss: 4.278862337589264. The RMSE of validation set is 4.687554747581482
Epoch [8/30]: The traing set Root Main Square Error loss: 4.1904711623191835. The RMSE of validation set is 4.572088578224

In [ ]:
LSTM with one hot

In [3]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.nn.functional as F
import numpy as np
import math
import pandas
from util.NNmodel.LSTM import TaxiLSTM, TaxiLSTM2
#from util.NNmodel.CNN import TaxiCNN
#import util.NNmodel.ConvLSTM 
from util.preprocess import TaxiDataset

#training config
epochs = 30
batch_size = 72
learning_rate=0.001

# Device config
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#train the model
def train_model(model, dataloader, criterion, optimizer):
    model.train()
    all_loss = []
    #for _ in tqdm(range(total_step), ncols=80):
    for i, (features, labels, locations, times) in enumerate(train_dataloader):
        features = features.to(torch.float32).to(device)
        labels = labels.to(torch.float32).to(device)
        locations = locations.to(torch.int64).to(device)
        times = times.to(torch.int64).to(device)

        # Forward pass
        outputs = model(features, locations, times)
        loss = torch.sqrt(criterion(outputs, labels))
        all_loss.append(loss.item())

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    return np.mean(all_loss)

# Validate the model performance
def val_model(model, dataloader, criterion):
    model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
    all_loss = []
    with torch.no_grad():
        for features, labels, locations, times in dataloader:
            features = features.to(torch.float32).to(device)
            labels = labels.to(torch.float32).to(device)
            locations = locations.to(torch.int64).to(device)
            times = times.to(torch.int64).to(device)

            outputs = model(features, locations, times)
            loss = torch.sqrt(criterion(outputs, labels)).item()
            all_loss.append(loss)
        return np.mean(all_loss)
    
    
# make prediction
def test_model(model, dataloader):
    model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
    with torch.no_grad():
        for features, labels, locations, times in dataloader: # one iteration
            features = features.to(torch.float32).to(device)
            locations = locations.to(torch.int64).to(device)
            times = times.to(torch.int64).to(device)

            outputs = model(features, locations, times)
            outputs = outputs.squeeze()
            p_array = outputs.detach().cpu().numpy()
            return np.rint(p_array)

def npz_parse(file):
    data = np.load(file)
    features = data['x']#features
    locations = data['locations']#locations
    times = data['times']#times
    if 'y' in data.files:
        labels = data['y']#labels
    else:
        labels = np.empty([len(features),1])
    return features, locations, times, labels

#read data
train_features, train_locations, train_times, train_labels = npz_parse('./train.npz')
val_features, val_locations, val_times, val_labels = npz_parse('./val.npz')
test_features, test_locations, test_times, test_labels = npz_parse('./test.npz')

# dataset
train_dataset = TaxiDataset(train_features, train_labels, train_locations, train_times)
val_dataset = TaxiDataset(val_features, val_labels, val_locations, val_times)
test_dataset = TaxiDataset(test_features, test_labels, test_locations, test_times)
# dataloader
train_dataloader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=len(test_features))

# initialize model
model = TaxiLSTM2().to(device)

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
for epoch in range(epochs):
    train_loss = train_model(model, train_dataloader, criterion, optimizer)
    val_loss = val_model(model, val_dataloader, criterion)
    print(f'Epoch [{epoch+1}/{epochs}]: The training set Root Main Square Error loss: {np.mean(train_loss)}. The RMSE of validation set is {val_loss}')

# prediction
pred_labels = test_model(model, test_dataloader)

# save prediction
np.savez('./test_pred.npz', x=test_features, y=pred_labels, locations=test_locations, times=test_times)
#with open('./time.txt',"a") as f:
#    for n in times:
#        f.write(str(n))
#    f.close()

Epoch [1/30]: The training set Root Main Square Error loss: 7.54881878733635. The RMSE of validation set is 6.896897667884827
Epoch [2/30]: The training set Root Main Square Error loss: 5.246568659305573. The RMSE of validation set is 5.429841484069824
Epoch [3/30]: The training set Root Main Square Error loss: 4.8946089286804195. The RMSE of validation set is 5.075543483734131
Epoch [4/30]: The training set Root Main Square Error loss: 4.616327150583267. The RMSE of validation set is 4.9745111665725705
Epoch [5/30]: The training set Root Main Square Error loss: 4.565713928937912. The RMSE of validation set is 4.937308276176453
Epoch [6/30]: The training set Root Main Square Error loss: 4.39887059879303. The RMSE of validation set is 4.766410461425782
Epoch [7/30]: The training set Root Main Square Error loss: 4.291275088787079. The RMSE of validation set is 4.744171131134033
Epoch [8/30]: The training set Root Main Square Error loss: 4.253871654510498. The RMSE of validation set is 5.